In [1]:
# %load query.py
import numpy as np
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
import requests

In [80]:
# %load query.py
import numpy as np
import glob

from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle
from astropy.time import Time

In [64]:
files = glob.glob('../IPTA_DR1/*/*.par')

In [105]:
RAs = []
DECs = []
Names = []
pmra = []
pmdec = []
pepoch = []
posepoch = []
dmepoch = []
dm = []
coords = []

for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            
    try:
        ra = Angle(d['RAJ'], u.hourangle).to(u.deg)
        dec = Angle(d['DECJ'], u.degree)
        coord = SkyCoord(ra=ra, dec=dec, frame='icrs',
                     pm_ra_cosdec = float(d['PMRA'])*np.cos(dec.to(u.rad))*u.mas/u.yr, 
                     pm_dec=float(d['PMDEC'])*u.mas/u.yr,
                     obstime=Time(float(d['POSEPOCH']), format='mjd'),
                     distance=1. * u.kpc) # HACK
        coords.append(coord)
        RAs.append(ra.value)
        #print(d['PSRJ'], d['DECJ'])
        DECs.append(dec.value)
        Names.append(d['PSRJ'])
        pmra.append(coord.pm_ra_cosdec.value)
        pmdec.append(coord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        lon = float(d['ELONG'])*u.deg
        lat = float(d['ELAT'])*u.deg
        coord = SkyCoord(l=lon, b=lat, frame='galactic',
                         pm_l_cosb = float(d['PMELONG'])*np.cos(lat.to(u.rad))*u.mas/u.yr, 
                         pm_b=float(d['PMELAT'])*u.mas/u.yr,
                         obstime=Time(float(d['POSEPOCH']), format='mjd'),
                         distance=1. * u.kpc) # HACKITY HACK HACK
        icrscoord = coord.transform_to(ICRS)
        coords.append(icrscoord)
        #print(icrscoord)
        #print(d['PSRJ'])
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append((icrscoord.pm_ra_cosdec/np.cos(icrscoord.dec.to(u.rad))).value)
        pmdec.append(icrscoord.pm_dec.value)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])        
        #coord = SkyCoord(d[''])
        
        continue
        
t = Table([Names, RAs, DECs, pmra, pmdec, pepoch, posepoch, dmepoch, dm], names=['names', 'ra', 'dec', 'pmra', 'pmdec', 'dm', 'pepoch', 'posepoch', 'dmepoch'])        
#tmin = Table([RAs, DECs], names=['ra', 'dec'])


In [111]:
print(coords[0].apply_space_motion(new_obstime=Time(57023., format='mjd')))

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (7.6142761, 4.86102859, 999.99999999)
 (pm_ra, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (-5.84650195, -0.41012739, 4.34121835e-06)>


In [114]:
gaia_time = Time(57023., format='mjd')

In [122]:
t['ra'] = [c.apply_space_motion(new_obstime=gaia_time).ra.value for c in coords]
t['dec'] = [c.apply_space_motion(new_obstime=gaia_time).dec.value for c in coords]

In [124]:
t.write('ipta_data.csv', format='csv', overwrite=True)
#tmin.write('ipta_data_nonames.csv', format='csv', overwrite=True)

In [125]:
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 1.0, 'RESPONSEFORMAT': 'csv',
'cat2': 'vizier:I/337/gaia', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open('ipta_data.csv', 'r')})

In [126]:
print(r.text)

angDist,names,ra,dec,pmra,pmdec,dm,pepoch,posepoch,dmepoch,ra_ep2000,dec_ep2000,errHalfMaj,errHalfMin,errPosAng,ra,dec,source_id,ref_epoch,ra_error,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b
0.267538,J0437-4715,69.3164670790048,-47.252667899419315,82.42957794891906,-71.47620158262019,54500,54500,54500,2.644571992220118,69.3165765588,-47.2526670522,0.003,0.002,0,69.3165765588,-47.2526670522,4789864076732331648,2015.0,1.992,2.741,,,,,,,-0.671,0,65,1.28203e+02,2.46615e+00,20.255,NOT_AVAILABLE,253.3946170111,-41.9631552362
0.005646,J1012+5307,153.13932707788018,53.117266431112,1.5746961099009713,-25.458808920674365,55000,55000,55000,9.021803950562022,153.1393295862,53.1172668707,0.001,0.000,0,153.1393295862,53.1172668707,851610857095609984,2015.0,0.395,0.614,,,,,,,0.058,0,148,2.43759e+02,1.93173e+00,19.557,NOT_AVAILABLE,160.3471938161,50.8578532

In [127]:
h = open('ipta_crossmatch_1arcsecond.csv', 'w')
h.write(r.text)
h.close()